<a href="https://colab.research.google.com/github/linhoangce/tensorflow_learning_stuff/blob/main/tensorflow_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Tensorflow variable is the recommended way to represent shared, persistent state our program manipulates.

Variables are created and tracked via the `tf.Variable` class. A `tf.Variable` represents a tensor whose value can be changed by running ops on it. Specific ops allow us to read and modify the values of this tensor. Higher level libraries like `tf.keras` use `tf.Variable` to store model parameters.

In [1]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

# Create a variable

In [2]:
my_tensor = tf.constant([[1., 2.],
                         [3., 4.]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


A variable looks and acts like a tensor, and, in fact, is a data structure backed by `tf.Tensor`. Like tensors, they have a `dtype` and a shape, and can be exported to NumPy.

In [3]:
my_variable.shape

TensorShape([2, 2])

In [4]:
my_variable.dtype

tf.float32

In [5]:
my_variable.numpy()

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0


array([[1., 2.],
       [3., 4.]], dtype=float32)

Most tensor operations work on variables as expected, although variables cannot be reshaped.

In [6]:
tf.convert_to_tensor(my_variable)

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

In [7]:
print(f"Index of highest values: {tf.math.argmax(my_variable)}")

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ArgMax in device /job:localhost/replica:0/task:0/device:GPU:0
Index of highest values: [1 1]


In [8]:
# This creates a new tensor, it does not reshape the variable
print("copying and reshaping: ", tf.reshape(my_variable, [1, 4]))

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
copying and reshaping:  tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


Creating a new variable from existing duplicates the backing tensors. Two variables will not share the same memory.

In [9]:
a = tf.Variable([2., 3.])

# Create b based on the value of a
b = tf.Variable(a)
a.assign([5., 6.])

# a and b are different
print(a.numpy())
print(b.numpy())

# Other assign versions
print(a.assign_add([2., 3.]).numpy())
print(a.assign_sub([7., 9.]).numpy())

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
[5. 6.]
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
[2. 3.]
Execut

## Lifecycle, naming, and watching

In Python-based TensorFlow, `tf.Variable` instance have the same lifecyle as other Python objects. When there are no references to a variable it is automatically deallocated.

Variables can also be named which can help track and debug them. Can have the same name

In [11]:
# Two tensors having the same name but backed by different tensors.
a = tf.Variable(my_tensor, name="Mark")
b = tf.Variable(my_tensor + 1, name="Mark")

# They are element-wise equal, but not identical
print(a == b)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Equal in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


# Placing variables and tensors

For better performance, TensorFlow will attempt to place tensors and variables on the fastest device compatible with its `dtype`. This means most variables are placed on a GPU if one is available.


In [12]:
with tf.device('CPU:0'):

  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1., 2.], [3., 4.], [5., 6.]])
  c = tf.matmul(a, b)

print(c)


Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


It's possible to set the location of a variable or tensor on one decice and do the computation on another device. This will introduce delay, as data needs to be copied between the device.

In [14]:
with tf.device('CPU:0'):
  a = tf.Variable([[1., 2., 3],
                   [4., 5., 6.]])
  b = tf.Variable([[1., 2., 3.]])

with tf.device('GPU:0'):
  k = a * b

k

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [ 4., 10., 18.]], dtype=float32)>